# Load Erp Data

Find and format File

erpdata=pd.read_csv('../1-AD/3-DUNS/duns_denormalized.csv',sep='|',encoding='utf-8',
                    dtype={
                        'companyname':str,
                          'countrycode':str,
                          'countryname':str,
                           'street':str,
                           'city':str,
                          'postalcode':str,
                          'registerid':str,
                          'source_level':str,
                            'source_relation':str,
                          'isglobalroot':bool})

In [8]:
erpdata=pd.read_excel('erpexcel.xlsx')

In [9]:
erpdata.dropna(subset=['companysystemid'],inplace=True)
erpdata.fillna('notavailable',inplace=True)

In [10]:
import neatmartinet as nm
for c in ['dunsid']:
    erpdata[c]=erpdata[c].apply(lambda r:nm.convert_int_to_str(n=r,fillwithzeroes=9))

In [11]:
erpdata.to_csv('../../Neo4j/sourcingreport.graphdb/import/erpdata_commasep.csv',encoding='utf-8',sep=',',index=False)

# Load in neo4j

### Connect to graph

In [12]:
import py2neo
graph = py2neo.Graph(user='neo4j',password='admin')

#### Load Erp

In [13]:
query_uploaderp='''
USING PERIODIC COMMIT 10000
  LOAD CSV WITH HEADERS FROM 'file:///erpdata_commasep.csv' AS row
    MERGE (c:ErpID{companysystemid: row.companysystemid})
    ON CREATE SET
        c.companysystemid=row.companysystemid,
        c.systemid=row.systemid,
        c.companylocalid=row.companylocalid,
        c.dunsid=row.dunsid,
        c.companyname=row.companyname,
        c.countrycode=row.countrycode,
        c.countryname=row.countryname,
        c.street=row.street,
        c.city=row.city,
        c.postalcode=row.postalcode,
        c.registerid=row.registerid,
        c.spent=toFloat(row.spent),
        c.creationdate=timestamp(),
        c.lastseen=timestamp()
    ON MATCH SET
        c.lastseen=timestamp(),
        c.spent=toFloat(row.spent);
'''
a=graph.run(query_uploaderp)
print(a.stats())

{'constraints_added': 0, 'constraints_removed': 0, 'indexes_added': 0, 'indexes_removed': 0, 'labels_added': 0, 'labels_removed': 0, 'nodes_created': 0, 'nodes_deleted': 0, 'properties_set': 100, 'relationships_deleted': 0, 'relationships_created': 0, 'contains_updates': True}


#### Create Relations

In [14]:
query_erphasduns='''
MATCH (a:ErpID),(b:DunsID)
WHERE b.dunsid = a.dunsid
MERGE (a)-[:hasDuns]->(b)
'''
a=graph.run(query_erphasduns)
print(a.stats())

{'constraints_added': 0, 'constraints_removed': 0, 'indexes_added': 0, 'indexes_removed': 0, 'labels_added': 0, 'labels_removed': 0, 'nodes_created': 0, 'nodes_deleted': 0, 'properties_set': 0, 'relationships_deleted': 0, 'relationships_created': 0, 'contains_updates': False}


# TO DO : 
- Create Country relations
- Write relation for RegisterId